# 1. Zapewne pamiętasz o odkurzaczu omówionym w pierwszych dwóch wykładach.
(i) Sformułuj problem jako problem przeszukiwania; tzn. musisz napisać stan
początkowy, możliwe działania za pomocą funkcji Action(state), relację
przejścia za pomocą funkcji Result(state, action) i stan celu. Proszę zobacz
yć strona 9 wykładu 2. Zakładamy, że obie kwadraty A i B są brudne i
agent znajduje się w kwadracie A. Zakładamy również, że agent ma trzy

możliwe działania: ruszaj w lewo, ruszaj w prawo i ssać. (ii) Napisz pro-
gram za pomocą algorytm ‘Breadth First Search, aby agent mógł odszukiwać

brud we wszystkich możliwych lokalizacjach i wyczyścić go (tzn., znalezienie
ścieżkę z węzła początkowego do węzła docelowego. Wystarczy, aby program
wydrukował każdą sekwencję akcji na konsoli, i pokazał kiedy osiąga cel
(czyli ścieżka z węzła początkowego do węzła docelowego).

In [7]:
from collections import deque

def find_shortest_cleaning_sequence(initial_state):
    # Initialize the queue with starting state and empty path
    queue = deque()
    queue.append((initial_state, []))
    
    # Track visited states to avoid cycles
    visited = set()
    
    while queue:
        current_state, path = queue.popleft()
        
        # Create a unique key for the state to check if we've seen it before
        state_key = (
            tuple(sorted(current_state['cleanliness'].items())),
            current_state['vacuum']
        )
        
        # Skip if we've already seen this state
        if state_key in visited:
            continue
            
        # Check if we've reached the goal (all positions clean)
        if all(status == 'clean' for status in current_state['cleanliness'].values()):
            return path
            
        # Mark this state as visited
        visited.add(state_key)
        
        # Generate all possible next states
        current_pos = current_state['vacuum']
        positions = current_state['positions']
        
        # Action 1: Clean current position
        if current_state['cleanliness'][current_pos] == 'dirty':
            new_state = current_state.copy()
            new_state['cleanliness'] = current_state['cleanliness'].copy()
            new_state['cleanliness'][current_pos] = 'clean'
            queue.append((new_state, path + ['clean']))
        
        # Action 2: Move left (to previous position in the list)
        current_index = positions.index(current_pos)
        if current_index > 0:
            new_pos = positions[current_index - 1]
            new_state = current_state.copy()
            new_state['vacuum'] = new_pos
            queue.append((new_state, path + ['move_left']))
        
        # Action 3: Move right (to next position in the list)
        if current_index < len(positions) - 1:
            new_pos = positions[current_index + 1]
            new_state = current_state.copy()
            new_state['vacuum'] = new_pos
            queue.append((new_state, path + ['move_right']))
    
    return None 

In [8]:
initial_state = {
    'positions': ['A', 'B'],  # Can add more positions like ['A', 'B', 'C']
    'cleanliness': {'A': 'dirty', 'B': 'dirty'},
    'vacuum': 'A'
}

solution = find_shortest_cleaning_sequence(initial_state)
print("Shortest cleaning sequence:", solution)

Shortest cleaning sequence: ['clean', 'move_right', 'clean']
